In [85]:
import os
import csv

In [86]:
threshold = 0.5
diff_threshold = 0.017
max_diff = 0.0
root_dir = '/home/drevital/obstacles_classification_datasets/model_eval'
model_name = 'kfs_jul22_d'
csv_path = os.path.join(root_dir, model_name+'.csv')
im_dir = os.path.join(root_dir, model_name + '_' + str(threshold))
tp_dir = os.path.join(im_dir, 'true_positives')
tn_dir = os.path.join(im_dir, 'true_negatives')
fp_dir = os.path.join(im_dir, 'false_positives')
fn_dir = os.path.join(im_dir, 'false_negatives')
predictions = {'imname': [], 'class': [], 'prediction': []}

In [87]:
with open(csv_path) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        predictions['imname'].append(row['Image Name'])
        predictions['class'].append(row['Class'])
        predictions['prediction'].append(float(row['Prediction']))

In [88]:
reader.line_num

197

In [89]:
tp_diff = 0.0
tn_diff = 0.0
fp_diff = 0.0
fn_diff = 0.0

## per category stats

In [90]:
orig_tp_imnames = os.listdir(tp_dir)
orig_tn_imnames = os.listdir(tn_dir)
orig_fp_imnames = os.listdir(fp_dir)
orig_fn_imnames = os.listdir(fn_dir)
tp_imnames = []
tn_imnames = []
fp_imnames = []
fn_imnames = []
tp_preds = []
tn_preds = []
fp_preds = []
fn_preds = []

for imname in orig_tp_imnames:
    lst = imname.split('_')
    pred = lst.pop(2)
    tp_imnames.append('_'.join(lst))
    tp_preds.append(float(pred))

for imname in orig_tn_imnames:
    lst = imname.split('_')
    pred = lst.pop(2)
    tn_imnames.append('_'.join(lst))
    tn_preds.append(float(pred))

for imname in orig_fp_imnames:
    lst = imname.split('_')
    pred = lst.pop(2)
    fp_imnames.append('_'.join(lst))
    fp_preds.append(float(pred))

for imname in orig_fn_imnames:
    lst = imname.split('_')
    pred = lst.pop(2)
    fn_imnames.append('_'.join(lst))
    fn_preds.append(float(pred))

tp_diff = 0.0
tn_diff = 0.0
fp_diff = 0.0
fn_diff = 0.0
tp_count = len(tp_imnames)
tn_count = len(tn_imnames)
fp_count = len(fp_imnames)
fn_count = len(fn_imnames)

In [91]:
trt_ind = 0
diff_imnames = []
for imname in predictions['imname']:
    if imname in tp_imnames:
        tf_ind = tp_imnames.index(imname)
        diff = abs(predictions['prediction'][trt_ind] - tp_preds[tf_ind])
        max_diff = max(diff, max_diff)
        tp_diff += diff
        if diff > diff_threshold:
            diff_imnames.append([imname, predictions['prediction'][trt_ind], tp_preds[tf_ind]])
    elif imname in tn_imnames:
        tf_ind = tn_imnames.index(imname)
        diff = abs(predictions['prediction'][trt_ind] - tn_preds[tf_ind])
        max_diff = max(diff, max_diff)
        tn_diff += diff
        if diff > diff_threshold:
            diff_imnames.append([imname, predictions['prediction'][trt_ind], tn_preds[tf_ind]])
    elif imname in fp_imnames:
        tf_ind = fp_imnames.index(imname)
        diff = abs(predictions['prediction'][trt_ind] - fp_preds[tf_ind])
        max_diff = max(diff, max_diff)
        fp_diff += diff
        if diff > diff_threshold:
            diff_imnames.append([imname, predictions['prediction'][trt_ind], fp_preds[tf_ind]])
    elif imname in fn_imnames:
        tf_ind = fn_imnames.index(imname)
        diff = abs(predictions['prediction'][trt_ind] - fn_preds[tf_ind])
        max_diff = max(diff, max_diff)
        fn_diff += diff
        if diff > diff_threshold:
            diff_imnames.append([imname, predictions['prediction'][trt_ind], fn_preds[tf_ind]])

    trt_ind += 1

In [92]:
tp_avg_diff = tp_diff / tp_count
tn_avg_diff = tn_diff / tn_count
#fp_avg_diff = fp_diff / fp_count
#fn_avg_diff = fn_diff / fn_count

In [93]:
tp_avg_diff, tn_avg_diff, fp_avg_diff, fn_avg_diff

(0.003544650629002561,
 0.0041723307371139525,
 0.08185663316276048,
 0.2772750778992971)

In [94]:
len(diff_imnames)

1

In [95]:
diff_imnames

[['39_obstacles_classification_2022-07-26T08-20-35_0.21.jpg',
  0.4225959777832031,
  0.44]]

In [96]:
len(diff_imnames)

1

In [97]:
max_diff

0.017404022216796877